In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\opdar\\BVM\\Mtech\\iiitb\\2nd Sem\\SPE\\End_project\\Object_Counting_1'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    export_model_path: Path

In [4]:
from src.Object_Counting.constants import *
from src.Object_Counting.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pt",
            training_data="dataset.yaml",
            export_model_path="torchserve/models/detection_1.onnx",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [6]:
from ultralytics import YOLO

In [13]:
import shutil

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    @staticmethod
    def load_model(path: Path) -> YOLO:
        return YOLO(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.results =  self.model.val(data=self.config.training_data, imgsz=self.config.params_image_size,batch=self.config.params_batch_size)

    def export_model(self):
        self.model.export(format="onnx")
        shutil.copy("artifacts/training/model.onnx", self.config.export_model_path)
        # shutil.copy("artifacts/training/detection.onnx", self.config.export_model_path)

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.results.results_dict)


In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.export_model()

except Exception as e:
   raise e

[2024-04-04 01:56:55,548: INFO: common: yaml file: C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\config\config.yaml loaded successfully]
[2024-04-04 01:56:55,550: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-04 01:56:55,551: INFO: common: created directory at: artifacts]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.14 torch-2.2.2+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\artifacts\data_ingestion\dataset\valid\labels.cache... 90 images, 0 backgrounds, 0 corrupt: 100%|██████████| 90/90 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.45it/s]


                   all         90       1502      0.535      0.365      0.341      0.202
               bicycle         90         30      0.142     0.0667     0.0709     0.0308
                   bus         90          4          1          0          0          0
                   car         90       1030      0.715      0.898      0.897      0.571
             motorbike         90        400      0.492       0.33       0.31      0.136
                person         90         38      0.325      0.532      0.426      0.272
Speed: 0.6ms preprocess, 26.8ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\val18
[2024-04-04 01:57:04,763: INFO: common: json file saved at: scores.json]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.14 torch-2.2.2+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)

PyTorch: starting from 'artifacts\training\model.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 9, 2100) (5.9 MB)

ONNX: starting export with onnx 1.15.0